In [2]:
!pip install gspread

     |████████████████████████████████| 152 kB 3.1 MB/s eta 0:00:01
     |████████████████████████████████| 155 kB 9.8 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 3.6 MB/s  eta 0:00:01


In [4]:
!pip install gspread_pandas

In [6]:
!pip install gsheetsdb

     |████████████████████████████████| 67 kB 2.0 MB/s eta 0:00:011
  Created wheel for moz-sql-parser: filename=moz_sql_parser-4.40.21126-py3-none-any.whl size=75208 sha256=e7fdb6e63a343f795069a02751b4c9ad22cf3c639eb7f70e06e6ba4a75136914
  Stored in directory: /home/jovyan/.cache/pip/wheels/13/00/86/b49607a5b82fd02e7eea1558f3dbb1f8dc02690de926f18de5
  Created wheel for mo-dots: filename=mo_dots-4.22.21108-py3-none-any.whl size=19819 sha256=b16e685d10a70c8a9afe7d6460e516bc5423e733413219f1fa5147894d52e776
  Stored in directory: /home/jovyan/.cache/pip/wheels/93/b8/1c/8e71a4e6464944525aea97290a19f26ade5109c4be1cbaa29c
  Created wheel for mo-future: filename=mo_future-3.147.20327-py3-none-any.whl size=4516 sha256=710308ee42bdd73b0e9a0bedf8d465b3afe838c1ffd140e5a21f2ca66738ae02
  Stored in directory: /home/jovyan/.cache/pip/wheels/6b/0a/2b/d053b1795c67d7e9adee25508b7ae65087705834cccdb5a94f
  Created wheel for mo-imports: filename=mo_imports-3.149.20327-py3-none-any.whl size=5089 sha256=374

In [1]:
import streamlit as st
from google.oauth2 import service_account

scopes=[
    "https://www.googleapis.com/auth/spreadsheets",
]

# Create a connection object.
credentials = service_account.Credentials.from_service_account_info(
    st.secrets["gcp_service_account"],
    scopes=scopes,
)

2021-08-10 11:00:51.490 INFO    numexpr.utils: NumExpr defaulting to 8 threads.


In [2]:
from gspread_pandas import Spread, Client

client = Client(scope=scopes, creds=credentials)
spread = Spread(st.secrets["private_gsheets_url"], client=client)

Your credentials don't have Drive API access, ignoring drive specific functionality (Note this includes searching spreadsheets by name)


In [3]:
test_df = spread.sheet_to_df(header_rows=None, index=None)

test_df

,0,1
0,A,B
1,1,a
2,2,b
3,3,c


In [42]:
df = pd.DataFrame({'A':[1,2,3], 'B':['a', 'b', 'c']})
df

,A,B
0,1,a
1,2,b
2,3,c


In [44]:
spread.df_to_sheet(df, index=False)

# Data merge

In [147]:
import pandas as pd
import streamlit as st
from google.oauth2 import service_account
from gspread_pandas import Spread, Client

class DataMerger():
    def __init__(self):
        scopes=[
            "https://www.googleapis.com/auth/spreadsheets",
        ]

        credentials = service_account.Credentials.from_service_account_info(
            st.secrets["gcp_service_account"],
            scopes=scopes,
        )
        
        client = Client(scope=scopes, creds=credentials)
        self.spread = Spread(st.secrets["private_gsheets_url"], client=client)
        self.columns = ['Transaction date', 'Description', 'Amount', 'Booked balance', 'Account', 'User']
        
    def cast(self, df):
        df[self.columns[0]] = pd.to_datetime(df[self.columns[0]], format='%Y-%m-%d').astype(str)
        df[self.columns[1]] = df[self.columns[1]].astype(str)
        df[self.columns[2:4]] = df[self.columns[2:4]].astype(float)
        df[self.columns[4:]] = df[self.columns[4:]].astype(str)
        return df
    
    def merge(self, tmp_df):
        master_df = self.spread.sheet_to_df(header_rows=1, index=None)
        master_df = self.cast(master_df)
        merged_df = pd.concat([master_df, tmp_df], ignore_index=True)
        merged_df = merged_df.drop_duplicates()
        merged_df = merged_df.sort_values(by=self.columns[0])
        self.spread.df_to_sheet(merged_df, index=False)
        
    def swedbank(self, path, name):
        tmp_df = pd.read_excel(path, header=7)
        tmp_df = tmp_df[self.columns[:4]]
        tmp_df[self.columns[4]] = 'Swedbank'
        tmp_df[self.columns[5]] = name
        tmp_df = self.cast(tmp_df)
        self.merge(tmp_df)

    def prestia(self, path, name):
        tmp_df = pd.read_csv(path, header=None, encoding='Shift-JIS')
        tmp_df = tmp_df.iloc[:,[0,2]]
        tmp_df.columns = [col for col in self.columns if col == 'Transaction date' or col == 'Amount']
        tmp_df[self.columns[0]] = tmp_df[self.columns[0]].str.replace('/', '-')
        tmp_df[self.columns[2]] = tmp_df[self.columns[2]].str.replace('SEK', '')
        tmp_df[self.columns[2]] = tmp_df[self.columns[2]].str.replace(',', '')
        tmp_df[self.columns[1]] = 'Not available'
        tmp_df[self.columns[3]] = 0
        tmp_df[self.columns[4]] = 'Prestia'
        tmp_df[self.columns[5]] = name
        tmp_df = self.cast(tmp_df)
        self.merge(tmp_df)
        
    def skandia(self, path, name):
        tmp_df = pd.read_excel(path, header=3)
        tmp_df.columns = self.columns[:4]
        tmp_df[self.columns[4]] = 'Skandia'
        tmp_df[self.columns[5]] = name
        tmp_df = self.cast(tmp_df)
        self.merge(tmp_df)
        
    def sony(self, path, name):
        tmp_df = pd.read_excel(path, header=0)
        tmp_df[['お預け入れ額', 'お引き出し額']] = tmp_df[['お預け入れ額', 'お引き出し額']].astype(float)
        tmp_df[['お預け入れ額', 'お引き出し額']] = tmp_df[['お預け入れ額', 'お引き出し額']].fillna(0)
        tmp_df = tmp_df.rename(columns={k:v for k,v in zip(['お取り引き日', '摘要', 'お預け入れ額', '差し引き残高'], self.columns[:4])})
        tmp_df[self.columns[0]] = pd.to_datetime(tmp_df[self.columns[0]], unit='D', origin='1899-12-30')
        tmp_df[self.columns[2]] = tmp_df[self.columns[2]] - tmp_df['お引き出し額']
        tmp_df[self.columns[4]] = 'Sony'
        tmp_df[self.columns[5]] = name
        tmp_df = tmp_df[self.columns]
        tmp_df = self.cast(tmp_df)
        self.merge(tmp_df)

In [149]:
dm = DataMerger()

dm.swedbank('./data/Swedbank_Transactions.xlsx', 'susumu')

Your credentials don't have Drive API access, ignoring drive specific functionality (Note this includes searching spreadsheets by name)


In [150]:
dm.prestia('./data/ACCT_252 (1).csv', 'susumu')

In [151]:
dm.skandia('./data/9159-916.791-5_2021-04-01-2021-06-21.xlsx', 'susumu')

In [152]:
dm.sony('./data/YenFutsuRireki-2019-2021-2-28.xlsx', 'hiromi')